In [1]:
import torch
import torch.nn as nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import torch.nn.functional as F

import random
import time
from datetime import timedelta
import shutil

import numpy as np

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pytorch_model_summary import summary

from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import multiprocessing
import timm

from PIL import Image

import os
import json

import pandas as pd
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('./dataset/train/hypothesis_valid.csv')

In [9]:
len(df)

3780

### mask ensemble effect

In [23]:
mask1_diff = pd.read_csv('./checkpoints/mask1/diff.csv')
mask2_diff = pd.read_csv('./checkpoints/mask2/diff.csv')
mask3_diff = pd.read_csv('./checkpoints/mask3/diff.csv')
mask_error_img1 = df[mask1_diff['labels']!=mask1_diff['preds']]['path'].tolist()
mask_error_img2 = df[mask2_diff['labels']!=mask2_diff['preds']]['path'].tolist()
mask_error_img3 = df[mask3_diff['labels']!=mask3_diff['preds']]['path'].tolist()
print(len(mask_error_img1), len(mask_error_img2), len(mask_error_img3))
total_error_case = []
total_error_case.extend(mask_error_img1)
total_error_case.extend(mask_error_img2)
total_error_case.extend(mask_error_img3)
pd.Series(total_error_case).value_counts() # 총 4개가 틀리개 됨

4 17 5


/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/006511_male_Asian_19/mask3.jpg               3
/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/006269_female_Asian_20/incorrect_mask.jpg    2
/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/006006_female_Asian_18/mask5.jpg             2
/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/006397_male_Asian_19/mask5.jpg               2
/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/005536_female_Asian_39/normal.jpg            1
/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/005473_female_Asian_51/mask2.jpg             1
/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/005459_male_Asian_60/incorrect_mask.jpg      1
/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/006098_

### gender ensemble effect

In [38]:
gender_diff1 = pd.read_csv('./checkpoints/gender_cls1/diff.csv')
gender_diff2 = pd.read_csv('./checkpoints/gender_cls2/diff.csv')
gender_diff3 = pd.read_csv('./checkpoints/gender_cls3/diff.csv')
gender_error_img1 = df[gender_diff1['labels']!=gender_diff1['preds']]['path'].tolist()
gender_error_img2 = df[gender_diff2['labels']!=gender_diff2['preds']]['path'].tolist()
gender_error_img3 = df[gender_diff3['labels']!=gender_diff3['preds']]['path'].tolist()
print(len(gender_error_img1), len(gender_error_img2), len(gender_error_img3))
total_error_case = []
total_error_case.extend(gender_error_img1)
total_error_case.extend(gender_error_img2)
total_error_case.extend(gender_error_img3)
bool_tag = pd.Series(total_error_case).value_counts()>=2 # 총 4개가 틀리개 됨
bool_tag.value_counts() # 총 92개 틀리게 됨

105 118 122


False    103
True      92
dtype: int64